In [1]:
#!/usr/bin/python
"""Set up and run simulations in Dymola, and then plot the results.
"""
__author__ = "Filip Jorissen"
__version__ = "2017-01-23"

import os
import matplotlib.pyplot as plt
import numpy as np
import re

from matplotlib import gridspec
from modelicares import gen_experiments, Experiment, SimResList, write_script, saveall

def merge_gens(gen1, gen2):
    result = []
    for i in gen1:
        result.append(i)
    for i in gen2:
        result.append(i)
    return result

def formatPlot(ax):
    ax.spines["top"].set_visible(False)    
    ax.spines["bottom"].set_visible(True)    
    ax.spines["right"].set_visible(False)    
    ax.spines["left"].set_visible(True)   
    ax.get_xaxis().tick_bottom()    
    ax.get_yaxis().tick_left() 

def plott(sim, name, coeffTime=1, coeffVar=1, offsetVar=0, offsetTime=0, ran=[0], **args):
    var = sim[name]
    time = sim["Time"]
    if ran == [0]:
        plt.plot((time.values()[0] + offsetTime)*coeffTime,(var.values()[0]+offsetVar)*coeffVar, **args)
    else:
        plt.plot((time.values()[0][ran] + offsetTime)*coeffTime,(var.values()[0][ran]+offsetVar)*coeffVar, **args)


In [2]:
# Based on code from Kevin Davies - ModelicaRes
# Name of the Modelica script (may include the path)
FNAME = 'run_sims.mos'

# Working directory
WORKING_DIR = '/tmp/scripts'

# List of Modelica packages that should be preloaded (besides the Modelica
# Standard Library)
# Each may be a *.mo file or a path where a package.mo file resides, e.g.,
# "/opt/dymola/Modelica/Library/VehicleInterfaces 1.1.1".
#PACKAGES = ["/home/filipjorissen/dymola.mos"]

# Using dymola.mos instead of named startup script
PACKAGES = [""]   

FIGPATH="/media/psf/Home/Documents/Software/IDEAS-git/IDEAS/Resources/Images/TwinHouse/"

start = 8e6;
end = 1.23e7;

values = 5000;


# List or generator of simulations to run
EXPERIMENTS=gen_experiments(models=["IDEAS.Examples.TwinHouses.BuildingN2_Exp2"],
                             args=dict(startTime=[start],stopTime=[end], method=['\"LSodar\"'], numberOfIntervals=[values]))


# Formats in which to save the figures (e.g., ['pdf', 'eps', 'svg', 'png'])
# If the figures shouldn't be saved, specify an empty list.
FORMATS = ['pdf', 'eps']

MODELS, RESULTS_DIR = write_script(EXPERIMENTS, working_dir=WORKING_DIR,
                                       packages=PACKAGES, fname=FNAME)		
                                       
                                    






In [46]:
# For Linux:
# need to add :
# Evaluate=true;
# Advanced.EfficientMinorEvents=true;
# to run_sims.mos script for getting fast simulation!
os.system('dymola ' + FNAME)

0

In [3]:
import matplotlib.dates as md


sim = SimResList(os.path.join(RESULTS_DIR, str(1), 'dsres.mat'))
fig=plt.figure(figsize=(10,10))

ranStart=1160
ran=range(ranStart,5000)

plotStart= - md.epoch2num(1388530800)*3600*24

rangeStart=-plotStart/3600/24 + sim["struct.Living.TAir"].times()[0][ranStart]/3600/24


color1="#C64E54"
color2="#4C74AB"
color3="#4CB391"
color4="#FF7D34"
color5="#A0AB7F"
color6="#333333"

legendsize=10

gs = gridspec.GridSpec(4,1,height_ratios=[1,1,1,2])


ax0 = plt.subplot(gs[0])    
plt.setp(ax0.get_xticklabels(), visible=False)
formatPlot(ax0)
plott(sim, "heaSys.heaSche.Pel_IDEAL[1]", lw=0.5, ls='--', dashes=(3, 1), ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="M", color=color1)
plott(sim, "heaSys.Qhea[1]", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="S", color=color2)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102),frameon=False,loc=2,ncol=2,prop={'size':legendsize})
plt.ylim([0,2500])
plt.ylabel("Radiator thermal \n power [W]")


ax1 = plt.subplot(gs[1], sharex=ax0)    
formatPlot(ax1)
plt.setp(ax1.get_xticklabels(), visible=False)
plott(sim, "validationDataExp2_1.living_h010cm_AT", lw=0.5, ls='--', dashes=(3, 1), ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="M: h = 10 cm", color=color1)
plott(sim, "validationDataExp2_1.living_h110cm_AT", lw=0.5, ls='--', dashes=(3, 1), ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="M: h = 110 cm", color=color2)
plott(sim, "validationDataExp2_1.living_h170cm_AT", lw=0.5, ls='--', dashes=(3, 1), ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="M: h = 170 cm", color=color3)
plott(sim, "struct.Living.TAir", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="S", color=color4)
# plt.xlim([0,800/24])
plt.ylim([20,42])
plt.ylabel("Air temperature \n at height h [$^{\circ}$C]")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102),frameon=False,loc=2,ncol=2,prop={'size':legendsize})


ax2 = plt.subplot(gs[2], sharex=ax0)    
plt.setp(ax2.get_xticklabels(), visible=False)
formatPlot(ax2)
plott(sim, "struct.W30.gainDir.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="North dir", color=color6)
plott(sim, "struct.W30.gainDif.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="North dif", color=color5)
plott(sim, "struct.W31.gainDir.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="South dir", color=color1)
plott(sim, "struct.W31.gainDif.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="South dif", color=color2)
plott(sim, "struct.W32.gainDir.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="West dir", color=color3)
plott(sim, "struct.W32.gainDif.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="West dif", color=color4)
plt.ylabel("Exterior solar \n irradiation [W]")
# plt.xlim([0,800/24])
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102),frameon=False,loc=2,ncol=3,prop={'size':legendsize})



ax3 = plt.subplot(gs[3], sharex=ax0)    
# plt.setp(ax0.get_xticklabels(), visible=False)
formatPlot(ax3)
plott(sim, "validationDataExp2_1.living_h110cm_AT", lw=0.5, ls='--', dashes=(4, 2), color=color1,  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: air 110 cm")
plott(sim, "struct.heatPortCon[1].T", lw=0.5, color=color1,  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: air")
plott(sim, "validationDataExp2_1.living_westfacade_S_IS_T", lw=0.5, color=color2, ls='--', dashes=(4, 2), ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: inner surface")
plott(sim, "struct.W9.layMul.port_a.T", lw=0.5, color=color2,  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: inner surface")
plott(sim, "validationDataExp2_1.living_westfacade_S_BL1_T", color=color3, lw=0.5, ls='--', dashes=(4, 2),  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: core")
plott(sim, "struct.W9.layMul.port_gain[3].T", color=color3, lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: core")
plott(sim, "validationDataExp2_1.living_westfacade_S_ES_T", lw=0.5, dashes=(4, 2), ls='--', color=color4,  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: outer surface")
plott(sim, "struct.W9.layMul.port_b.T", lw=0.5, color=color4,  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: outer surface")
plott(sim, "sim.Te", lw=0.5, ls='--', dashes=(4, 2), color=color6,  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: exterior")
plott(sim, "validationDataExp2_1.living_h170cm_AT", lw=0.5, ls='--', dashes=(4, 2), color=color5,  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: air 170 cm")
plt.ylabel("Wall section temperatures [$^{\circ}$C]")

# plt.xlim([0,800/24])
plt.ylim([0,45])
plt.yticks([0, 10, 20, 30, 40])
plt.legend(bbox_to_anchor=(0., 0.95, 1., .102),frameon=False,loc=2,ncol=5,prop={'size':legendsize})
plt.xlabel("Date")
plt.xlim([rangeStart,rangeStart+33])

ax=plt.gca()
ax.xaxis_date()
xfmt = md.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(xfmt)

plt.tight_layout()

plt.savefig(FIGPATH + "TwinHouse.eps")
plt.savefig(FIGPATH + "TwinHouse.pdf")

#plt.show()
